# Simple Decision Tree Classifier

In [5]:
import pandas as pd
import numpy as np
import os
import csv
from operator import itemgetter

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier

import nDCG

## Load dataset and choose features

In [41]:
# load dataset
df = pd.read_pickle('cleaned_dataset')

# add column with relevance scores
df['relevance'] = df['booking_bool'] + df['click_bool']

# change relevance column with relevance score (1 if clicked, 5 if booked)
df['relevance'] = df['relevance'].map({0:0, 1:1, 2:5})

# select features to train model with
selected_features = ['value_for_money','price_rank', 'star_diff_from_historic_mean','comp1_rate_percent_diff',\
                     'srch_query_affinity_score','srch_room_count','price_diff_from_historic_mean', \
                      'visitor_hist_adr_usd', 'site_id', 'prop_review_score', 'prop_location_score1']
# selected_features = ['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', \
#                      'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score',\
#                      'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', \
#                      'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',\
#                      'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', \
#                      'srch_query_affinity_score', 'orig_destination_distance', 'srch_weekday', 'srch_month', \
#                      'srch_quarter', 'srch_year', 'book_start_weekday', 'book_start_month', 'book_start_quarter', \
#                      'book_start_year', 'book_end_weekday', 'book_end_month', 'book_end_quarter', 'book_end_year', \
#                      'comp', 'comp1', 'comp2', 'comp3', 'comp4', 'comp5', 'comp6', 'comp7', 'comp8', \
#                      'avg_prop_starrating', 'avg_prop_location_score1', 'avg_prop_location_score2', 'avg_price_usd',\
#                      'avg_prop_starrating_diff', 'avg_prop_location_score1_diff', 'avg_prop_location_score2_diff', \
#                      'avg_price_usd_diff']

# selected_features = ['site_id', 'visitor_location_country_id',
#        'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
#        'prop_starrating', 'prop_review_score', 'prop_brand_bool',
#        'prop_location_score1', 'prop_location_score2',
#        'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
#        'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
#        'srch_adults_count', 'srch_children_count', 'srch_room_count',
#        'srch_saturday_night_bool', 'srch_query_affinity_score',
#        'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
#        'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
#        'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
#        'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
#        'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
#        'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
#        'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
#        'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
#        'comp8_rate_percent_diff', 'click_bool', 'gross_bookings_usd',
#        'price_diff_from_historic_mean', 'star_diff_from_historic_mean', 
#        'price_rank', 'star_rank','value_for_money', 'price_usd_normalized']

features_to_keep = ['relevance', 'srch_id', 'prop_id', 'booking_bool']
all_possible_features = selected_features + features_to_keep
df_select = df.loc[:,all_possible_features]

In [42]:
df_select=df_select.dropna()
df_select

,value_for_money,price_rank,star_diff_from_historic_mean,comp1_rate_percent_diff,srch_query_affinity_score,srch_room_count,price_diff_from_historic_mean,visitor_hist_adr_usd,site_id,prop_review_score,prop_location_score1,relevance,srch_id,prop_id,booking_bool
0,0.033407,3.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,3.5,2.83,0,1,893,0
1,0.023427,14.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.0,2.20,0,1,10404,0
2,0.025028,15.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.5,2.20,0,1,21315,0
3,0.006656,22.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.0,2.83,0,1,27348,0
4,0.024377,11.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,3.5,2.64,0,1,29604,0
5,0.023039,19.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.5,2.77,0,1,30184,0
6,0.027058,8.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,3.5,2.20,0,1,44147,0
7,0.000000,1.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,0.0,1.61,0,1,50984,0
8,0.026658,12.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.0,2.56,0,1,53341,0
9,0.014251,21.0,-150.94,10.0,-20.4591,1,-25.28,154.27,12,4.0,2.83,0,1,56880,0


## Create training and test set taking into account search IDs

In [43]:
# create list of ids in random order
id_list = df_select.srch_id.unique()
np.random.shuffle(id_list)

# for reproducibility 
np.random.seed(1337)

# training - test fraction is 70/30
id_train = int(len(id_list) * 0.7)
training_id = id_list[:id_train]
test_id = id_list[id_train:]

# divide dataset in test and training set
train_data = df_select.loc[df.srch_id.isin(training_id)]
test_data = df_select.loc[df.srch_id.isin(test_id)]

## Training the model

In [44]:
# select features and target variable
features = train_data[selected_features].values
target = train_data["relevance"].values

# train randomforest classifier on training data
classifier = RandomForestClassifier(n_estimators=50, 
                                    verbose=2,
                                    n_jobs=1,
                                    min_samples_split=10,
                                    random_state=1)
classifier.fit(features, target)

building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.9min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=1, verbose=2, warm_start=False)

## Predict test set

In [45]:
def write_submission(recommendations, submission_file):
    """
    Function which writes submission, ordered on the probability obtained by the model.
    The columns are SearchId, PropertyId and Relevance    
    """
    submission_path = "/home/marleen/projects/DM2018/Assignment2/Bench_Results/"+submission_file
    rows = [(srch_id, prop_id, relevance)
        for srch_id, prop_id, relevance, rank_float
        in sorted(recommendations, key=itemgetter(0,3))]
    writer = csv.writer(open(submission_path, "w"), lineterminator="\n")
    writer.writerow(("SearchId", "PropertyId", "Relevance"))
    writer.writerows(rows)

In [46]:
# test model with features of test dataset
features = test_data[selected_features].values

# make predictions with trained model
predictions = classifier.predict_proba(features)[:,1]
predictions = list(-1.0*predictions)
recommendations = list(zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], predictions))

# write predictions to csv file
write_submission(recommendations, "results.csv")

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    6.2s finished


In [47]:
# compute nDCG for results of test set
path_results = "Bench_Results/results.csv"
print(nDCG.compute_ndcg(path_results))

0.407398774036


## Compare with random results

In [ ]:
# write file with random results
# np.random.seed(1)
ordinals = np.arange(len(test_data))
np.random.shuffle(ordinals)
recommendations = zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], ordinals)
write_submission(recommendations, "randomBenchmark.csv")

In [ ]:
# compute nDCG 
path_results = "Bench_Results/randomBenchmark.csv"
print(nDCG.compute_ndcg(path_results))

## Compare with perfect results

In [ ]:
# write file with perfect results
ordinals = np.arange(len(test_data))
predictions = np.array(test_data['relevance'])
predictions = list(-1*predictions)
recommendations = zip(test_data["srch_id"], test_data["prop_id"], test_data['relevance'], predictions)
write_submission(recommendations, "testOrderBenchmark.csv")

In [ ]:
# compute nDCG 
path_results = "Bench_Results/testOrderBenchmark.csv"
print(nDCG.compute_ndcg(path_results))